<a href="https://colab.research.google.com/github/rhedouin/Polymetric_landcover/blob/main/lulc_san_francisco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install -q earthengine-api geemap

In [ ]:
# Change this to your EE Project (find your project name at https://code.earthengine.google.com/)
EE_PROJECT='ee-eo4datascience'

In [ ]:
# Import required libraries
import ee
import geemap
from datetime import datetime

# Authenticate and initialize the Earth Engine API
try:
    ee.Initialize(project=EE_PROJECT)
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project=EE_PROJECT)

# Define the bounding box coordinates
bounding_box = ee.Geometry.Rectangle([
    -125.0004161, 34.9995834,  # Lower Left (longitude, latitude)
    -119.9995827, 40.0004167   # Upper Right (longitude, latitude)
])

# Load Google Earth Engine datasets
# Dynamic World dataset (Sentinel-2 derived)
dynamic_world = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')

# ESA WorldCover dataset (Land Cover)
esa_worldcover = ee.ImageCollection("ESA/WorldCover/v200")

# Define a date range for filtering the Dynamic World dataset
start_date = '2021-01-01'
end_date = '2021-12-31'

# Filter Dynamic World for the bounding box and time range
dw_filtered = dynamic_world.filterBounds(bounding_box).filterDate(start_date, end_date)

# Get the median land cover for Dynamic World
dw_median = dw_filtered.median().clip(bounding_box)

# Get the latest ESA Land Cover image
esa_image = esa_worldcover.filterBounds(bounding_box).first().clip(bounding_box)

# Export Dynamic World median image to Google Drive as a COG
dw_task = ee.batch.Export.image.toDrive(
    image=dw_median.toFloat(),
    description='Dynamic_World_COG',
    folder='EarthEngineExports',
    fileNamePrefix='Dynamic_World_LULC',
    region=bounding_box.getInfo()['coordinates'],
    scale=10,  # Adjust scale (spatial resolution) as required
    crs='EPSG:4326',
    fileFormat='GeoTIFF',
    formatOptions={
        'cloudOptimized': True
    },
    maxPixels=1e13  # Set the maximum number of pixels
)
dw_task.start()

# Export ESA Land Cover image to Google Drive as a COG
esa_task = ee.batch.Export.image.toDrive(
    image=esa_image.toFloat(),
    description='ESA_LandCover_COG',
    folder='EarthEngineExports',
    fileNamePrefix='ESA_LandCover',
    region=bounding_box.getInfo()['coordinates'],
    scale=10,  # Adjust scale (spatial resolution) as required
    crs='EPSG:4326',
    fileFormat='GeoTIFF',
    formatOptions={
        'cloudOptimized': True
    },
    maxPixels=1e13  # Set the maximum number of pixels
)
esa_task.start()

# Check the task status
print('Dynamic World export task:', dw_task.status())
print('ESA Land Cover export task:', esa_task.status())


Dynamic World export task: {'state': 'READY', 'description': 'Dynamic_World_COG', 'priority': 100, 'creation_timestamp_ms': 1733994161067, 'update_timestamp_ms': 1733994161067, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'Y7EIAPCMTTPFWA4E4IV7KUAB', 'name': 'projects/ee-eo4datascience/operations/Y7EIAPCMTTPFWA4E4IV7KUAB'}
ESA Land Cover export task: {'state': 'READY', 'description': 'ESA_LandCover_COG', 'priority': 100, 'creation_timestamp_ms': 1733994161783, 'update_timestamp_ms': 1733994161783, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JR7ADDR63OUY37I3PPVCTP26', 'name': 'projects/ee-eo4datascience/operations/JR7ADDR63OUY37I3PPVCTP26'}
